# Lane Detection with Yolov8 and OpenCV 

Código para desplegar las líneas de un carril vehicular a partir de los resultados del modelo de Yolov8 de segmentación de tres clases: línea derecha, centro (carril), línea izquierda utilizando la función de transformada de líneas de Hough.

## Importar Librerías

Setting the environment of python and the files that we are goint to use in te proyect.

Librerías:
* OpenCV - 4.6.0
* Numpy -
* Matplotlib -

Archivos:
* Test Image: carril_seg.jpg
* Test Video: carril_predict.mp4 

In [27]:
import cv2
import numpy as np
from ultralytics import YOLO

## Funciones para desplegar las líneas

**Dibujo de líneas en la imagen**

Original

In [2]:
def display_lines(image, lines):
    line_image = np.zeros_like(image)
    if lines is not None:
        for x1, y1, x2, y2 in lines:
            print("display_lines: ",(x1, y1, x2, y2))
            cv2.line(line_image, (x1, y1), (x2, y2), (255, 0, 0), 10)
    return line_image

Codigo perfeccionado

In [3]:
def display_lines(image, lines):
    line_image = np.zeros_like(image)
    if lines is not None:
        for x1, y1, x2, y2 in lines:
            print("display_lines: ",(x1, y1, x2, y2))
            cv2.line(line_image, (x1, y1), (x2, y2), (255, 0, 0), 10)
    return line_image

**Puntos para obtener una línea**

Original

In [17]:
def make_coordinates(image, line_parameters):
    # print(line_parameters)
    if np.isnan(line_parameters).any():
        print('No line parameters')
        return np.array([0, 0, 0, 0])
    else:
        slope, intercept = line_parameters
        y1 = image.shape[0]
        y2 = int(y1*(3/5))
        x1 = int((y1 - intercept)/slope)
        x2 = int((y2 - intercept)/slope)
        return np.array([x1, y1, x2, y2])

Perfeccionado

In [21]:
def make_coordinates(img_height, line_parameters):
    # print(line_parameters)
    if np.isnan(line_parameters).any():
        print('No line parameters')
        return np.array([0, 0, 0, 0])
    else:
        slope, intercept = line_parameters
        y1 = img_height
        y2 = int(y1*(3/5))
        x1 = int((y1 - intercept)/slope)
        x2 = int((y2 - intercept)/slope)
        if x1 > 10000 or x2 > 10000 or x1 < -10000 or x2 < -10000:
            return np.array([0, 0, 0, 0])
        return np.array([x1, y1, x2, y2])

**Puntos medios**

In [5]:
def center_pt(coordinates):    
    x = int((coordinates[0]+coordinates[2])/2)
    y = int((coordinates[1]+coordinates[3])/2)
    return np.array([x,y])

**Puntos medios - Dexter Lolo**

In [20]:
def center_pt(yPresent,yFuture,lines):
    x_coordinates = []
    cont = 0
    for line in lines:
        x1, y1, x2, y2 = line
        # Calculate slope
        if(x2==x1):
            m = 0.001
        else:
            m = (y2 - y1) / (x2 - x1)
        # Calculate x coordinate
        x = ((yPresent - y1) / m) + x1
        # print("I - slope and x - center pts",(m,x))
        x=int(x)
        x_coordinates.append(x)
        x = ((yFuture - y1) / m) + x1
        # print("II - slope and x - center pts",(m,x))
        x=int(x)
        x_coordinates.append(x)
        cont+=1
    #     print("Line- ",line)
    # print("CONTADOR - ",cont)
    if ((x_coordinates[2]-x_coordinates[0])%2==0):
        xPm=int(x_coordinates[0]+((x_coordinates[2]-x_coordinates[0])/2))
    else:
        xPm=int(x_coordinates[0]+((x_coordinates[2]-x_coordinates[0]+1)/2))
    if ((x_coordinates[3]-x_coordinates[1])%2==0):
        xFm=int(x_coordinates[1]+((x_coordinates[3]-x_coordinates[1])/2))
    else:
        xFm=int(x_coordinates[1]+((x_coordinates[3]-x_coordinates[1]+1)/2))
    centerPoints=np.array([
        np.array([xPm, yPresent]),
        np.array([xFm, yFuture])
    ])
    print("Center Points", centerPoints)
    return(centerPoints)

**Display Center Points**

In [6]:
def makePoints(image, centers):
    circle_image = np.zeros_like(image)
    if centers is not None:
        for x, y in centers:
            cv2.circle(circle_image, (x,y),1, (255, 255, 255), 5)
    return circle_image

**Promedio de las líneas**

In [6]:
# def average_slope_intercept(image, lines):
def average_slope_intercept(lines):
    left_fit = []
    right_fit = []
    for line in lines:
        x1, y1, x2, y2 = line.reshape(4)
        parameters = np.polyfit((x1, x2), (y1, y2), 1)
        slope = parameters[0]
        intercept = parameters[1]
        if slope < 0:
            left_fit.append((slope, intercept))
        else:
            right_fit.append((slope, intercept))
    # Promedio
    left_fit_average = np.average(left_fit, axis=0)
    right_fit_average = np.average(right_fit, axis=0)
    # Mediana
    # left_fit_average = np.median(left_fit, axis=0)
    # right_fit_average = np.median(right_fit, axis=0)
    # left_line = make_coordinates(image, left_fit_average)
    # right_line = make_coordinates(image, right_fit_average)
    # # print("right",right_line)
    # # print("Lft",left_line)
    # pt_up = center_pt(np.array([left_line[2],left_line[3],right_line[2],right_line[3]]))
    # pt_low = center_pt(np.array([left_line[0],left_line[1],right_line[0],right_line[1]]))
    # print("UP",pt_up)
    # print("LOW",pt_low)
    # return np.array([left_line, right_line])
    return np.array([left_fit_average, right_fit_average])

### Original

In [19]:
def average_slope_intercept(img_height, lines):
    left_fit = []
    right_fit = []
    for line in lines:
        x1, y1, x2, y2 = line.reshape(4)
        parameters = np.polyfit((x1, x2), (y1, y2), 1)
        slope = parameters[0]
        intercept = parameters[1]
        if slope < 0:
            # print("SI - Left",(slope, intercept))
            left_fit.append((slope, intercept))
        else:
            # print("SI - Right",(slope, intercept))
            right_fit.append((slope, intercept))
    # Promedio
    left_fit_average = np.average(left_fit, axis=0)
    right_fit_average = np.average(right_fit, axis=0)
    # Mediana
    # left_fit_average = np.median(left_fit, axis=0)
    # right_fit_average = np.median(right_fit, axis=0)
    # print("Left line average",left_fit_average)
    # print("Right line average",right_fit_average)
    left_line = make_coordinates(img_height, left_fit_average)
    right_line = make_coordinates(img_height, right_fit_average)
    return np.array([left_line, right_line])

## Despliege de los resultados en video - total

In [29]:
# model = YOLO("./Yolov8/weights/best_feb2024_FINSA.pt")  # FINSA segmentation model
model = YOLO("./Yolov8/weights/best_CampusSeg.pt")  # Campus segmentation model
names = model.model.names
print(names)

{0: 'Bicycle-road', 1: 'Campus-road', 2: 'Lane-road', 3: 'catsEye', 4: 'craks', 5: 'roadUnpaved', 6: 'stormDrain'}


**Código para pruebas - se despliegan los resultados de la segmentación en imagen binaria**

In [ ]:
model = YOLO("./Yolov8/weights_feb2024/best.pt")  # segmentation model
cap = cv2.VideoCapture("./Yolov8/carril.mp4")
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

out = cv2.VideoWriter('instance-segmentation-new-model.avi', cv2.VideoWriter_fourcc(*'MJPG'), fps, (w, h))
out_bin = cv2.VideoWriter('instance-segmentation-binary-new-model.avi', cv2.VideoWriter_fourcc(*'MJPG'), fps, (w, h))

names = model.model.names
while True:
    ret, im0 = cap.read()
    if not ret:
        print("Video frame is empty or video processing has been successfully completed.")
        break

    results = model.predict(im0)
    height,width = im0.shape[:2]
    polylines_im = np.zeros((height,width, 1), np.uint8)
    averaged_lines_left = []
    averaged_lines_right = []
    if results[0].masks is not None:
        clss = results[0].boxes.cls.cpu().tolist()
        masks = results[0].masks.xy
        for mask, cls in zip(masks, clss):
            if mask is not None and names[int(cls)] == "center":
                cv2.polylines(polylines_im, [np.int32(mask)], isClosed=False, color=255, thickness=5)
                cv2.polylines(im0, [np.int32(mask)], isClosed=False, color=(255,255,0), thickness=5)
    polylines_im = cv2.medianBlur(polylines_im, 7)
    height,width = polylines_im.shape[:2]
    polylines_im[height-200:height, 0:width] = 0
    cv2.imshow("binary", polylines_im)
    lines = cv2.HoughLinesP(polylines_im, 5, np.pi/180, 100, np.array([]), minLineLength=100, maxLineGap=10)
    if lines is not None:
        averaged_lines= average_slope_intercept(lines)
        print("avarage Left", averaged_lines[0])
        averaged_lines_left.append(averaged_lines[0])
        averaged_lines_right.append(averaged_lines[1])
        if averaged_lines[0] is None:
            averaged_lines_left.pop()
            averaged_lines[0] = averaged_lines_left[len(averaged_lines_left)-1]
        if averaged_lines[1] is None:
            averaged_lines_right.pop()
            averaged_lines[1] = averaged_lines_right[len(averaged_lines_right)-1]
        # Crear coordenadas de las futuras líneas
        left_line = make_coordinates(im0, averaged_lines[0])
        right_line = make_coordinates(im0, averaged_lines[1])
        line_image = display_lines(im0, np.array([left_line,right_line]))
        im0 = cv2.addWeighted(im0, 0.8, line_image, 1, 1)
        
    for line in lines:
        x1, y1, x2, y2 = line[0]
        cv2.line(im0, (x1, y1), (x2, y2), (0, 255, 0), 2)

    out.write(im0)
    out_bin.write(polylines_im)
    cv2.imshow("instance-segmentation", im0)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

out.release()
cap.release()
cv2.destroyAllWindows()

**Video con Dexter Lolo**

In [ ]:
cap = cv2.VideoCapture('./Yolov8/carril.mp4')
# w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))
# out = cv2.VideoWriter('segmentacion-feb2024.avi', cv2.VideoWriter_fourcc(*'MJPG'), fps, (w, h))
counter = 0 
flag = True
while(cap.isOpened()):
    counter +=1
    ret, im0 = cap.read()
    if not ret:
        print("Video frame is empty or video processing has been successfully completed.")
        break
    # Se realiza una copya a escala de grises de la imagen original para visualizar los resultados de forma más rápida
    im0_gray = np.copy(im0)
    im0_gray = cv2.cvtColor(im0_gray, cv2.COLOR_BGR2GRAY)
    
    # Se realiza la predicción de la imagen
    results = model.predict(im0)
    # Se crea una imagen binaria para visualizar las máscaras
    height,width = im0.shape[:2]
    polylines_im = np.zeros((height,width, 1), np.uint8)
    
    if results[0].masks is not None:
        clss = results[0].boxes.cls.cpu().tolist()
        masks = results[0].masks.xy
        for mask, cls in zip(masks, clss):
            if mask is not None and names[int(cls)] == "center":
                cv2.polylines(polylines_im, [np.int32(mask)], isClosed=True, color=255, thickness=5)
                # cv2.polylines(im0, [np.int32(mask)], isClosed=False, color=(255,255,0), thickness=5)
        # polylines_im = cv2.medianBlur(polylines_im, 7)
        # height,width = polylines_im.shape[:2]
        polylines_im[height-205:height, 0:width] = 0
        polylines_im[0:450, 0:width] = 0
        cv2.imshow("binary", polylines_im)
        lines = cv2.HoughLinesP(polylines_im, 5, np.pi/180, 100, np.array([]), minLineLength=100, maxLineGap=10)
        if lines is not None:
            averaged_lines = average_slope_intercept(height, lines)
            print("AVERAGED LINES", averaged_lines)
            if not np.isnan(averaged_lines).any():
                line_image = display_lines(im0_gray, averaged_lines)
                center_points = center_pt(500,680,averaged_lines)
                combo_image = cv2.addWeighted(im0_gray, 0.8, line_image, 1, 1)
                center_point_image=makePoints(combo_image,center_points)
                combo_combo_image=cv2.addWeighted(combo_image, 0.8, center_point_image, 1, 1)
            else:
                print("No line parameters", averaged_lines)
        if counter%2 == 0:    
            cv2.imshow('result',combo_combo_image)

    # out.write(combo_combo_image)
    if cv2.waitKey(1) == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

Version Jetson

In [26]:
cap = cv2.VideoCapture('/home/fcanof/vanttec_sdv/workspace/src/sdv_vision/yolov8_lane_detection/carril.mp4')
# w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))
# out = cv2.VideoWriter('segmentacion-feb2024.avi', cv2.VideoWriter_fourcc(*'MJPG'), fps, (w, h))
counter = 0 
flag = True
while(cap.isOpened()):
    counter +=1
    ret, im0 = cap.read()
    if not ret:
        print("Video frame is empty or video processing has been successfully completed.")
        break
    # Se realiza una copya a escala de grises de la imagen original para visualizar los resultados de forma más rápida
    im0_gray = np.copy(im0)
    im0_gray = cv2.cvtColor(im0_gray, cv2.COLOR_BGR2GRAY)
    combo_combo_image = im0_gray
    
    # Se crea una imagen binaria para visualizar las máscaras
    height,width = im0.shape[:2]
    polylines_im = np.zeros((height,width, 1), np.uint8)
    
    # Se realiza la predicción de la imagen
    results = model.predict(im0, classes=0)
    if counter%2 == 0:    
        if results[0].masks is not None:
            masks = results[0].masks.xy[0]
            cv2.polylines(polylines_im, [np.int32(masks)], isClosed=True, color=255, thickness=5)
            polylines_im[height-205:height, 0:width] = 0
            polylines_im[0:450, 0:width] = 0
            cv2.imshow("binary", polylines_im)
            lines = cv2.HoughLinesP(polylines_im, 5, np.pi/180, 100, np.array([]), minLineLength=100, maxLineGap=10)
            if lines is not None:
                averaged_lines = average_slope_intercept(height, lines)
                if not np.isnan(averaged_lines).any():
                    center_points = center_pt(500,680,averaged_lines)
                    for x1, y1, x2, y2 in averaged_lines:
                        cv2.line(im0_gray, (x1, y1), (x2, y2), (255, 0, 0), 10)
                    for x, y in center_points:
                        cv2.circle(im0_gray, (x,y),1, (255, 0, 0), 5)
    cv2.imshow('result',im0_gray)

    # out.write(combo_combo_image)
    if cv2.waitKey(1) == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

0: 384x640 1 center, 8.6ms
Speed: 15.3ms preprocess, 8.6ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 center, 8.6ms
Speed: 1.8ms preprocess, 8.6ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)
Center Points [[633 500]
 [671 680]]

0: 384x640 1 center, 9.3ms
Speed: 1.8ms preprocess, 9.3ms inference, 2.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 center, 11.9ms
Speed: 2.2ms preprocess, 11.9ms inference, 2.6ms postprocess per image at shape (1, 3, 384, 640)
Center Points [[629 500]
 [650 680]]

0: 384x640 1 center, 9.3ms
Speed: 1.8ms preprocess, 9.3ms inference, 2.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 center, 11.6ms
Speed: 4.7ms preprocess, 11.6ms inference, 2.6ms postprocess per image at shape (1, 3, 384, 640)
Center Points [[623 500]
 [641 680]]

0: 384x640 1 center, 14.2ms
Speed: 2.1ms preprocess, 14.2ms inference, 3.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 c

**Código final**

In [ ]:
# model = YOLO("./Yolov8/weights_train4/best.pt")  # segmentation model - past
model = YOLO("./Yolov8/weights_feb2024/best.pt")  # segmentation model - past
names = model.model.names
cap = cv2.VideoCapture("./Yolov8/carril.mp4")
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

out = cv2.VideoWriter('instance-segmentation-20.avi', cv2.VideoWriter_fourcc(*'MJPG'), fps, (w, h))

while True:
    ret, im0 = cap.read()
    if not ret:
        print("Video frame is empty or video processing has been successfully completed.")
        break

    results = model.predict(im0)

    polylines_im = np.zeros((im0.shape[0], im0.shape[1], 1), np.uint8)
    height,width = polylines_im.shape[:2]
    if results[0].masks is not None:
        clss = results[0].boxes.cls.cpu().tolist()
        masks = results[0].masks.xy
        for mask, cls in zip(masks, clss):
            if mask is not None and names[int(cls)] == "center":
                cv2.polylines(polylines_im, [np.int32(mask)], isClosed=False, color=255, thickness=5)
    polylines_im = cv2.medianBlur(polylines_im, 7)
    polylines_im[height-100:height, 0:width] = 0
    lines = cv2.HoughLinesP(polylines_im, 5, np.pi/180, 100, np.array([]), minLineLength=100, maxLineGap=10)
    if lines is not None:
        averaged_lines = average_slope_intercept(im0, lines)
        line_image = display_lines(im0, averaged_lines)
        im0 = cv2.addWeighted(im0, 0.8, line_image, 1, 1)

    out.write(im0)
    cv2.imshow("instance-segmentation", im0)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

out.release()
cap.release()
cv2.destroyAllWindows()

Codigo optimizado por la jetson

In [ ]:
  def listener_callback(self, data):
    """
    Callback function.
    """
    # Display the message on the console
    self.get_logger().info('Receiving video frame')

    # Convert ROS Image message to OpenCV image
    current_frame = self.br.imgmsg_to_cv2(data)
    counter = self.counter + 1
    msg = Float64MultiArray()

    # Auxiliar images to display
    frame_gray = np.copy(current_frame)
    frame_gray = cv2.cvtColor(frame_gray, cv2.COLOR_BGR2GRAY)
    # combo_combo_image = frame_gray
    height,width = current_frame.shape[:2]
    polylines_im = np.zeros((height, width, 1), np.uint8)

    # YOLO predictions
    results = self.MODEL.predict(current_frame)
    if counter%2 == 0:    
      if results[0].masks is not None:
          clss = results[0].boxes.cls.cpu().tolist()
          masks = results[0].masks.xy
          for mask, cls in zip(masks, clss):
              if mask is not None and self.MODEL_NAMES[int(cls)] == "center":
                  cv2.polylines(polylines_im, [np.int32(mask)], isClosed=False, color=255, thickness=5)
          polylines_im[height-205:height, 0:width] = 0
          polylines_im[0:450, 0:width] = 0
          lines = cv2.HoughLinesP(polylines_im, 5, np.pi/180, 100, np.array([]), minLineLength=100, maxLineGap=10)
          if lines is not None:
              averaged_lines = average_slope_intercept(height, lines)
              if not np.isnan(averaged_lines).any():
                # line_image = display_lines(im0_gray, averaged_lines)
                center_points = center_pt(500,680,averaged_lines)
                for x1, y1, x2, y2 in averaged_lines:
                    # print("display_lines: ",(x1, y1, x2, y2))
                    cv2.line(frame_gray, (x1, y1), (x2, y2), (255, 0, 0), 10)
                for x, y in center_points:
                    cv2.circle(frame_gray, (x,y),1, (255, 0, 0), 5)
                # combo_image = cv2.addWeighted(im0_gray, 0.8, line_image, 1, 1)
                # center_point_image=makePoints(combo_image,center_points)
                # combo_combo_image=cv2.addWeighted(combo_image, 0.8, center_point_image, 1, 1)
                # Publish center points
                msg.data = center_points.flatten()
                self.publisher_center_pts.publish(msg)

    cv2.imshow('result',frame_gray)
    cv2.waitKey(1)